I have hourly data for geographical positions, speed over goround, and CO2 emissions from 100 unique container ships for the year 2019. I have also loaded a list of major ports across the world. Assuming the sample is complete and representative, I intened to present a simple analysis of air pollution around ports. 

In [2]:
import pandas as pd
import sqlite3
import folium

Reading the data:

In [7]:
con = sqlite3.connect("dwh/db")

ships = pd.read_sql_query('select * from ships', con)
ports = pd.read_sql_query('select * from ports', con)


I prepared data for visualisation with a SQL query to and load an aggregated table with ports and total emissions to the database.


In [8]:
ports_emissions_query = """
    SELECT s.closest_port, s.continent, SUM(s.E_CO2_kg) AS total_E_CO2_kg, p.LATITUDE, p.LONGITUDE
    FROM ships s
    LEFT JOIN ports p ON s.closest_port = p.PORT_NAME
    WHERE s.closest_port IS NOT NULL
    GROUP BY s.closest_port
"""
ports_emissions = pd.read_sql_query(ports_emissions_query, con)


# Load ports_emissions into the database
ports_emissions.to_sql('ports_emissions', con, if_exists='replace', index=False)

564

In [9]:
ports_emissions = pd.read_sql_query('select * from ports_emissions', con)

I used this data to visualise the geographic distribution of relative CO2 emissions from container shipping from this smaple(assuming the sample is representative). This is done through a simple method via the folium library, which requires a location for the center of the map, the coordinates, the values for emissions, and allows adjustments for visual representation.

In [10]:

m = folium.Map(location=[20, 0], tiles="OpenStreetMap", zoom_start=2)

for index, row in ports_emissions.iterrows():
    folium.CircleMarker(
        location=[row['LATITUDE'], row['LONGITUDE']],
        radius=row['total_E_CO2_kg'] / 1000000, 
        popup=f"{row['closest_port']} - {row['total_E_CO2_kg']:.2f} kg",
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7
    ).add_to(m)

m


what is the share of emissions at port compared to emissons at sea?

I  was curious to know if ships are turning off their engine at port. This is interesting for carbon analysis, as shorepower energy (or 'cold ironing' in maritime terms) is a known solution for decarbonisation.